In [4]:
from datetime import datetime
import pandas_datareader as reader

df = reader.get_data_yahoo(['BTC-USD'], start=datetime(2015,7,30))
df


Attributes     Adj Close         Close          High           Low  \
Symbols          BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2015-07-29    287.721985    287.721985    290.126007    286.567993   
2015-07-30    284.649994    284.649994    288.959015    282.343994   
2015-07-31    281.601013    281.601013    284.932007    278.112000   
2015-08-01    282.614014    282.614014    283.032013    277.528992   
2015-08-02    281.226990    281.226990    285.471008    280.233002   
...                  ...           ...           ...           ...   
2021-11-06  61527.480469  61527.480469  61590.683594  60163.781250   
2021-11-07  63326.988281  63326.988281  63326.988281  61432.488281   
2021-11-08  67566.828125  67566.828125  67673.742188  63344.066406   
2021-11-09  66971.828125  66971.828125  68530.335938  66382.062500   
2021-11-10  68409.648438  68409.648438  68744.031250  66377.671875   

Attributes          Open        Volume  
Symbols          BTC-USD       BTC-USD  
Date                                    
2015-07-29    289.102997  2.163580e+07  
2015-07-30    287.696014  2.362910e+07  
2015-07-31    284.686005  1.899500e+07  
2015-08-01    280.449005  1.772220e+07  
2015-08-02    282.806000  2.147410e+07  
...                  ...           ...  
2021-11-06  61068.875000  2.909493e+10  
2021-11-07  61554.921875  2.472675e+10  
2021-11-08  63344.066406  4.112561e+10  
2021-11-09  67549.734375  4.235799e+10  
2021-11-10  67089.476562  3.911657e+10  

[2292 rows x 6 columns]

In [5]:
import pandas as pd

pd.plotting.register_matplotlib_converters()

print(df.info())

df['Close'].plot(figsize=(16,6))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2292 entries, 2015-07-29 to 2021-11-10
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   (Adj Close, BTC-USD)  2292 non-null   float64
 1   (Close, BTC-USD)      2292 non-null   float64
 2   (High, BTC-USD)       2292 non-null   float64
 3   (Low, BTC-USD)        2292 non-null   float64
 4   (Open, BTC-USD)       2292 non-null   float64
 5   (Volume, BTC-USD)     2292 non-null   float64
dtypes: float64(6)
memory usage: 125.3 KB
None


In [6]:
df[df.index.duplicated()]

Attributes     Adj Close         Close          High           Low  \
Symbols          BTC-USD       BTC-USD       BTC-USD       BTC-USD   
Date                                                                 
2016-03-27    424.230988    424.230988    426.856995    423.292999   
2017-03-26   1045.770020   1045.770020   1046.400024    971.984009   
2018-03-25   8209.400391   8209.400391   8530.080078   7921.430176   
2019-03-31   4158.183105   4158.183105   4164.953125   4096.901367   
2020-03-29   6429.841797   6429.841797   6517.195801   5903.234375   
2021-03-28  57750.199219  57750.199219  58342.097656  55139.339844   

Attributes          Open        Volume  
Symbols          BTC-USD       BTC-USD  
Date                                    
2016-03-27    426.548004  6.852280e+07  
2017-03-26    972.054993  3.725350e+08  
2018-03-25   8498.469727  5.921040e+09  
2019-03-31   4105.362305  1.015779e+10  
2020-03-29   5925.538574  3.710165e+10  
2021-03-28  55947.898438  5.762559e+10

In [7]:
df = df[~df.index.duplicated(keep='first')]
df = df['Close']

In [8]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

df_sans_last_60_days = df[:-60]
df_sans_last_60_days.reset_index(inplace=True)
training_data = Dataset.Tabular.register_pandas_dataframe(
    df_sans_last_60_days, datastore, 'BTCRate')

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/data/dataset_factory.py:1216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_datetime(df[column])


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/45d44c65-50f7-4173-a561-f29966be36c9/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_name = 'shrek'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print(f'Using existing compute: {compute_name}')    
else:
    print(f'Creating new compute: {compute_name}')
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size = 'Standard_F4s_v2',
        min_nodes = 0, 
        max_nodes = 4)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    compute_target.wait_for_completion(show_output=True)

Creating new compute: shrek
InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(
    time_column_name='Date',
    freq='1D',
    forecast_horizon=1,
    feature_lags='auto',
    target_lags=[7],
    use_stl=None,
    country_or_region_for_holidays='US',
)

In [9]:
automl_config = AutoMLConfig(
    task = "forecasting",
    primary_metric='normalized_root_mean_squared_error',

    training_data = training_data,
    label_column_name = 'BTC-USD',
    n_cross_validations=7,

    enable_early_stopping=True,
    early_stopping_n_iters=20,
    max_concurrent_iterations=4,
    iteration_timeout_minutes=5,

    compute_target=compute_target,
    forecasting_parameters=forecasting_parameters
)

In [10]:
from azureml.core import Experiment
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
experiment = Experiment(ws, "AutoML-Forecasting")
run = experiment.submit(automl_config, show_output=True)


best_run, best_model = run.get_output()

Submitting remote run.
No run_configuration provided, running on shrek with default configuration
Running on remote compute: shrek


In [33]:
from azureml.core import Experiment
from azureml.train.automl.run import AutoMLRun

ws = Workspace.from_config()
datastore = ws.get_default_datastore()

experiment = Experiment(ws, "AutoML-Forecasting")
runs = list(experiment.get_runs()) 

run = AutoMLRun(experiment, runs[0].id)
best_run, best_model = run.get_output()

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


KeyboardInterrupt: 

In [ ]:
from datetime import datetime

range = pd.date_range(datetime(2021,10,10), datetime(2021,10,20))

forecast_df = pd.DataFrame({ 'Date': range })

In [ ]:
forecast, transformed_df = best_model.forecast(forecast_df)

In [ ]:
pretty_df = forecast_df.copy().set_index('Date')
pretty_df['ETH-USD-forecast'] = forecast
pretty_df['ETH-USD'] = df['ETH-USD']

pretty_df[['ETH-USD', 'ETH-USD-forecast']].plot(figsize=(16,6))